## pyomo - operation room scheduling optimization
### Concrete model
- **decision variables, objective function, constraints**

### Problem 
The task centers on optimizing the scheduling of operating rooms in a healthcare facility over a three-month span. The objective is to maximize the number of procedures accommodated within this period. The model focuses on assigning procedures to available operating rooms while taking into account the duration of each procedure, room availability, and compatibility constraints. By identifying an optimal schedule, the aim is to improve resource utilization and maximize the number of procedures, thereby enhancing operational efficiency.
#### Decision variable
**Schedule:** A binary decision variable used in the scheduling model to indicate whether a procedure is scheduled. It takes a value of 1 if the procedure is scheduled and 0 otherwise. This variable is indexed by Encounter ID, Operating Room (OR) Suite, day, and time, capturing whether a specific procedure is assigned to a given OR suite at a particular day and time.
#### Parameters
- **OR_Suites:** Refers to the available Operating Room Suites, totaling three in the facility.
- **CPT_Codes:** Represents the set of Current Procedural Terminology (CPT) codes, with a total of 32 distinct codes, each identifying a specific procedure type.
- **Booked_Time:** Indicates the estimated duration booked for each procedure, which varies by CPT Code.
- **Encounter ID:** A unique identifier assigned to each procedure request, totaling 2,172 requests. To streamline calculations, only 15% of these procedures will be selected through random assignment for the scheduling exercise.

#### Constraints
To structure the optimization model, the following constraints have been established:

- The cumulative booked time across all OR suites on any given day must not exceed 24 hours (based on the capacity of 3 OR Suites operating 8 hours each).
- Each OR Suite is limited to performing one procedure at any given time.
- Every procedure must be assigned to only one OR Suite.
- Each procedure should be scheduled exactly once within the planning period.
- The total weekly booked time for each OR Suite is capped at 56 hours.
- The booked time for any individual OR Suite on a single day is limited to a maximum of 8 hours. 

These constraints are designed to ensure feasible allocation of resources while adhering to time and operational limitations.

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pyomo.environ import *
# read csv file
df = pd.read_csv('2022_Q1_OR_Utilization.csv')
df.head() # prints the top 4 rows
df.describe() # prints the summary statistics


,index,Encounter ID,OR Suite,CPT Code,Booked Time (min)
count,2172.000000,2172.000000,2172.000000,2172.000000,2172.000000
mean,1085.500000,11086.500000,4.288674,44881.405617,77.189227
std,627.146713,627.146713,2.163514,18087.419079,30.430015
min,0.000000,10001.000000,1.000000,14060.000000,30.000000
25%,542.750000,10543.750000,3.000000,28296.000000,60.000000
50%,1085.500000,11086.500000,4.000000,42826.000000,60.000000
75%,1628.250000,11629.250000,6.000000,66982.000000,90.000000
max,2171.000000,12172.000000,8.000000,69436.000000,180.000000


In [89]:
print("shape:", df.shape)
print("count of unique encounter id:",df['Encounter ID'].nunique())
print("count of unique cpt code:",df['CPT Code'].nunique())
print("procedure having same time?",df.groupby('CPT Code')['Booked Time (min)'].nunique())


shape: (2172, 13)
count of unique encounter id: 2172
count of unique cpt code: 32
procedure having same time? CPT Code
14060    1
15773    1
17110    1
26045    1
26356    1
26735    1
27130    1
27445    1
28055    1
28060    1
28110    1
28285    1
28289    1
28296    1
28297    1
28820    1
29877    1
30400    1
30520    1
36901    1
42826    1
43775    1
47562    1
52353    1
55250    1
55873    1
57460    1
58562    1
64721    1
66982    2
69421    1
69436    1
Name: Booked Time (min), dtype: int64


To simplify the scheduling model, procedure times have been standardized for each individual CPT code. This means that every procedure associated with a specific CPT code will have a fixed duration, allowing for more streamlined calculations and consistency in the allocation of operating room time.

In [90]:
print("CPT Code 66982 booked time:", df[df['CPT Code'] == 66982]['Booked Time (min)'].unique()) # checking

CPT Code 66982 booked time: [45 30]


In [91]:
df.loc[df['CPT Code'] == 66982, 'Booked Time (min)'] = 45
print("CPT Code 66982 booked time:", df[df['CPT Code'] == 66982]['Booked Time (min)'].unique()) # check again
print("procedure having same time?",df.groupby('CPT Code')['Booked Time (min)'].nunique())

CPT Code 66982 booked time: [45]
procedure having same time? CPT Code
14060    1
15773    1
17110    1
26045    1
26356    1
26735    1
27130    1
27445    1
28055    1
28060    1
28110    1
28285    1
28289    1
28296    1
28297    1
28820    1
29877    1
30400    1
30520    1
36901    1
42826    1
43775    1
47562    1
52353    1
55250    1
55873    1
57460    1
58562    1
64721    1
66982    1
69421    1
69436    1
Name: Booked Time (min), dtype: int64


1) Dropping the not so needed columns from the dataframe
2) Transforming the Booked time which is in minutes to hours
3) changing the column name Booked Time (min) to Booked Time (hr)

In [92]:
df.drop(['Date', 'OR Suite', 'OR Schedule',	'Wheels In', 'Start Time', 'End Time', 'Wheels Out', 'Service'], axis=1, inplace=True)
df['Booked Time (min)'] = df['Booked Time (min)'] / 60
df.rename(columns={'Booked Time (min)': 'Booked Time (hr)'}, inplace=True)

### Data Reduction

To manage computational effort effectively, and given that the dataset represents 90 days of operations, this exercise will utilize a proportion of 15% of the data, equivalent to approximately six weeks. This reduction will allow the model to run within a reasonable timeframe while still providing a representative sample of the data for analysis.

In [93]:
df = df.sample(frac=0.15, random_state=1)
df.head() # prints the top 4 rows
df.shape # prints the shape of the dataframe

(326, 5)

In [94]:
print("Total time booked for all procedures:", df['Booked Time (hr)'].sum())

Total time booked for all procedures: 413.25


A Pyomo concrete model was implemented to optimize the scheduling of operating rooms within a healthcare facility over a three-month period. By defining the problem with decision variables, an objective function, and specific constraints, the goal was to maximize the number of scheduled procedures while respecting various operational limitations.

The total weekly hours available across the 3 Operating Rooms is 168 hours, based on 8 hours per day, 7 days a week, for all OR suites. The total "demand" hours, derived from the data frame, amount to 413 hours, which is 2.45 times the available hours.

In [95]:
# Define an optimization model to allocate Operating Room schedules over a 3-month period

# Key model assumptions:
# 1. Scheduling is planned on a weekly basis as a repeating cycle.
# 2. Operating Rooms are available all week, including weekends, with rotating teams.
# 3. Each Operating Room operates for 8 hours per day, with staff shifts from 8 AM to 5 PM.

model = ConcreteModel()

# Define sets for the model
model.ProcedureType = Set(initialize=df['CPT Code'].unique())
model.ProcedureID = Set(initialize=df['Encounter ID'].unique())
model.WeekDays = Set(initialize=range(1, 8))
model.WorkHours = Set(initialize=range(8, 17))
model.Rooms = Set(initialize=range(1, 4))

# Define parameters for procedure times and capacities
model.EstimatedTime = Param(model.ProcedureType, initialize=df.set_index('CPT Code')['Booked Time (hr)'].to_dict())
model.ProcedureMapping = Param(model.ProcedureID, initialize=df.set_index('Encounter ID')['CPT Code'].to_dict())
model.RoomDailyMax = Param(model.Rooms, initialize={1: 8, 2: 8, 3: 8})

# Define binary decision variables for scheduling
model.schedule = Var(model.ProcedureID, model.Rooms, model.WeekDays, model.WorkHours, domain=Binary)

# Define the objective function to maximize scheduled procedure time
def objective_function(model):
    return sum(model.schedule[proc, room, day, hour] * model.EstimatedTime[model.ProcedureMapping[proc]]
               for proc in model.ProcedureID for room in model.Rooms for day in model.WeekDays for hour in model.WorkHours)

# Objective declaration to maximize total scheduled time
model.obj = Objective(rule=objective_function, sense=maximize)

# Constraints

# Ensure the total scheduled time across all rooms does not exceed 24 hours per day
def max_daily_hours(model, day):
    return sum(model.schedule[proc, room, day, hour] * model.EstimatedTime[model.ProcedureMapping[proc]]
               for proc in model.ProcedureID for room in model.Rooms for hour in model.WorkHours) <= 24

# Ensure each room is limited to one procedure at any given time
def room_occupancy_constraint(model, room, day, hour):
    return sum(model.schedule[proc, room, day, hour] for proc in model.ProcedureID) <= 1

# Ensure each procedure is scheduled in one room at a time
def procedure_room_constraint(model, proc, day, hour):
    return sum(model.schedule[proc, room, day, hour] for room in model.Rooms) <= 1

# Ensure each procedure is only scheduled once during the week
def single_scheduling_constraint(model, proc):
    return sum(model.schedule[proc, room, day, hour] for room in model.Rooms for day in model.WeekDays for hour in model.WorkHours) <= 1

# Weekly capacity constraint: Total hours for each room cannot exceed 56 hours
def weekly_capacity_constraint(model, room):
    return sum(model.schedule[proc, room, day, hour] * model.EstimatedTime[model.ProcedureMapping[proc]]
               for proc in model.ProcedureID for day in model.WeekDays for hour in model.WorkHours) <= 56

# Daily capacity constraint for each room
def daily_room_limit(model, room, day):
    return sum(model.schedule[proc, room, day, hour] * model.EstimatedTime[model.ProcedureMapping[proc]]
               for proc in model.ProcedureID for hour in model.WorkHours) <= 8

# Add constraints to the model
model.daily_limit_constraint = Constraint(model.WeekDays, rule=max_daily_hours)
model.one_per_room_constraint = Constraint(model.Rooms, model.WeekDays, model.WorkHours, rule=room_occupancy_constraint)
model.unique_room_constraint = Constraint(model.ProcedureID, model.WeekDays, model.WorkHours, rule=procedure_room_constraint)
model.single_schedule_constraint = Constraint(model.ProcedureID, rule=single_scheduling_constraint)
model.weekly_capacity_constraint = Constraint(model.Rooms, rule=weekly_capacity_constraint)
model.daily_room_constraint = Constraint(model.Rooms, model.WeekDays, rule=daily_room_limit)

The GLPK solver was used to achieve an optimal solution, maximizing the number of procedures scheduled and adhering to all defined constraints. The results were organized into a clear table, displaying each scheduled procedure’s Encounter ID, operating room assignment, day, hour, booked time, and end time.

In [96]:
# Solve the model using the GLPK solver
solver = SolverFactory('glpk', executable='D:/UNIVERSITY/SEMESTER 7/OR/glpk-4.65/w64/glpsol')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 168.0, 'Upper bound': 168.0, 'Number of objectives': 1, 'Number of constraints': 21084, 'Number of variables': 61614, 'Number of nonzeros': 369684, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '2531', 'Number of created subproblems': '2531'}}, 'Error rc': 0, 'Time': 241.1559820175171}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [97]:
# Store scheduling results in a DataFrame
schedule_results = pd.DataFrame(columns=['Procedure ID', 'Operating Room', 'Day', 'Hour'])

# List to collect rows for the DataFrame
results_list = []

# Extract results from the model into a list of dictionaries
for proc in model.ProcedureID:
    for room in model.Rooms:
        for day in model.WeekDays:
            for hour in model.WorkHours:
                if model.schedule[proc, room, day, hour]() == 1:
                    results_list.append(
                        {'Procedure ID': proc, 'Operating Room': room, 'Day': day, 'Hour': hour}
                    )

# Convert list of results into a DataFrame
schedule_results = pd.DataFrame(results_list)


In [98]:
schedule_results

,Procedure ID,Operating Room,Day,Hour
0,11286,3,7,8
1,10702,1,2,11
2,11734,1,2,10
3,10135,3,7,13
4,10686,3,7,9
...,...,...,...,...
79,11970,3,5,11
80,11756,3,6,8
81,11732,3,6,9
82,11816,3,6,10


In [99]:
# Cross-reference scheduled times with initial data to get booked hours
schedule_results = pd.merge(schedule_results, df[['Encounter ID', 'Booked Time (hr)']], left_on='Procedure ID', right_on='Encounter ID', how='left')


In [100]:
schedule_results.head()

,Procedure ID,Operating Room,Day,Hour,Encounter ID,Booked Time (hr)
0,11286,3,7,8,11286,1.50
1,10702,1,2,11,10702,0.75
2,11734,1,2,10,11734,2.00
3,10135,3,7,13,10135,2.00
4,10686,3,7,9,10686,1.50


In [101]:
# Calculate total hours per room and day
total_hours_per_room = schedule_results.groupby('Operating Room')['Booked Time (hr)'].sum()
total_hours_per_day = schedule_results.groupby('Day')['Booked Time (hr)'].sum()
total_hours_per_room_day = schedule_results.groupby(['Operating Room', 'Day'])['Booked Time (hr)'].sum()
print("Total hours per room:\n", total_hours_per_room)
print("\nTotal hours per day:\n", total_hours_per_day)
print("\nTotal hours per room and day:\n", total_hours_per_room_day)

Total hours per room:
 Operating Room
1    56.0
2    56.0
3    56.0
Name: Booked Time (hr), dtype: float64

Total hours per day:
 Day
1    24.0
2    24.0
3    24.0
4    24.0
5    24.0
6    24.0
7    24.0
Name: Booked Time (hr), dtype: float64

Total hours per room and day:
 Operating Room  Day
1               1      8.0
                2      8.0
                3      8.0
                4      8.0
                5      8.0
                6      8.0
                7      8.0
2               1      8.0
                2      8.0
                3      8.0
                4      8.0
                5      8.0
                6      8.0
                7      8.0
3               1      8.0
                2      8.0
                3      8.0
                4      8.0
                5      8.0
                6      8.0
                7      8.0
Name: Booked Time (hr), dtype: float64


In [102]:
# Calculate end times for each procedure
schedule_results['End Time'] = schedule_results['Hour'] + schedule_results['Booked Time (hr)']

In [103]:
# Sort the schedule results by room, day, and hour for ordered viewing
schedule_results = schedule_results.sort_values(by=['Operating Room', 'Day', 'Hour'])

## Sample Schedule optimization for 3 OR suites on Day 1

In [104]:
# Display the schedule for Day 1 and Operating Room 1
schedule_results[(schedule_results['Operating Room'] == 1) & (schedule_results['Day'] == 1)]

,Procedure ID,Operating Room,Day,Hour,Encounter ID,Booked Time (hr),End Time
39,10127,1,1,8,10127,3.0,11.0
7,11267,1,1,9,11267,2.0,11.0
57,10921,1,1,10,10921,3.0,13.0


In [105]:
# Display the schedule for Day 1 and Operating Room 2
schedule_results[(schedule_results['Operating Room'] == 2) & (schedule_results['Day'] == 1)]

,Procedure ID,Operating Room,Day,Hour,Encounter ID,Booked Time (hr),End Time
28,11291,2,1,8,11291,2.0,10.0
29,11245,2,1,9,11245,2.0,11.0
30,10852,2,1,10,10852,2.0,12.0
31,11821,2,1,11,11821,2.0,13.0


In [106]:
# Display the schedule for Day 1 and Operating Room 3
schedule_results[(schedule_results['Operating Room'] == 3) & (schedule_results['Day'] == 1)]

,Procedure ID,Operating Room,Day,Hour,Encounter ID,Booked Time (hr),End Time
59,10276,3,1,8,10276,2.0,10.0
60,10422,3,1,9,10422,2.0,11.0
61,11178,3,1,10,11178,2.0,12.0
62,11643,3,1,11,11643,2.0,13.0


Through the use of Pyomo and a concrete model, the scheduling team can improve resource utilization and streamline the operating room scheduling process. This approach provides flexibility in addressing different constraints and objectives, making it highly suitable for similar scheduling challenges in healthcare settings.

## The End